In [2]:
import numpy             as np
import matplotlib.pyplot as plt
import scipy.sparse      as spa

import time

import sys; sys.path.insert(0, '..')

import hylife.utilitis_FEEC.basics.spline_evaluation_1d as eva_1d
import hylife.utilitis_FEEC.basics.spline_evaluation_2d as eva_2d
import hylife.utilitis_FEEC.basics.spline_evaluation_3d as eva_3d

import hylife.utilitis_FEEC.bsplines         as bsp 
import hylife.utilitis_FEEC.bsplines_kernels as bsp_ker
import hylife.utilitis_FEEC.Bspline          as bspline
import hylife.utilitis_FEEC.spline_space     as spl

## 1d

In [20]:
Nel        = 8                # number of elements on logical domain
bc         = True             # boundary conditions (True: periodic, False: else)
p          = 3                # spline degrees  

el_b       = np.linspace(0., 1., Nel + 1)                     # element boundaries
T          = bsp.make_knots(el_b, p, bc)                      # knot vectors (for N functions)

nq_el      = 6   # number of quadrature points per element for integrations over whole domain

space      = spl.spline_space_1d(T, p, bc, nq_el)       # 1d B-spline space

N          = bspline.Bspline(space.T, space.p)          # N-splines for comparison
D          = bspline.Bspline(space.t, space.p - 1)      # D-splines for comparison

In [25]:
# test basis_funs_all
for cases in range(10):
    eta    = np.random.rand()
    
    values = np.empty((p + 1, p + 1), dtype=float)
    diff   = np.empty( p            , dtype=float)
    
    span   = bsp_ker.find_span(T, p, eta)
    
    bsp_ker.basis_funs_all(T, p, eta, span, values, diff)
    
    bN = np.empty(p + 1, dtype=float)
    bD = np.empty(p    , dtype=float)
    
    for il in range(p + 1):
        i = span - p + il
        bN[il] = N(eta, i)
        
    for il in range(p):
        i = span - p + il
        bD[il] = D(eta, i)*p/(T[i + p + 1] - T[i + 1])
        
    print(np.allclose(bN, values[-1]))
    print(np.allclose(bD, values[-2, :-1]*diff))

True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True


In [26]:
values

array([[1.        , 0.        , 0.        , 0.        ],
       [0.28742358, 0.71257642, 0.        , 0.        ],
       [0.04130616, 0.70481127, 0.25388257, 0.        ],
       [0.00395745, 0.33981225, 0.59592671, 0.06030358]])

In [29]:
values[p - 1, :p]*diff

array([0.33044927, 5.63849014, 2.03106059])

In [17]:
# test basis_funs_and_der
for cases in range(10):
    eta    = np.random.rand()
    
    values = np.empty((2, p + 1), dtype=float)
    
    span   = bsp_ker.find_span(T, p, eta)
    
    bsp_ker.basis_funs_and_der(T, p, eta, span, values)
    
    bN    = np.empty(p + 1, dtype=float)
    bdiff = np.empty(p + 1, dtype=float)
    
    for il in range(p + 1):
        i = span - p + il
        bN[il] = N(eta, i)
        bdiff[il] = N(eta, i, 1)
        
    print(np.allclose(bN, values[0]))
    print(np.allclose(bdiff, values[1]))

True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True


In [6]:
# test evaluate_n
for cases in range(10):
    coeff = np.random.rand(space.NbaseN)

    value = 0.
    eta   = np.random.rand()

    for i in range(N.N):
        value += coeff[i%space.NbaseN]*N(eta, i)

    print(np.allclose(value, eva_1d.evaluate_n(space.T, space.p, space.NbaseN, coeff, eta)))

True
True
True
True
True
True
True
True
True
True


In [7]:
# test evaluate_d
for cases in range(10):
    coeff = np.random.rand(space.NbaseD)

    value = 0.
    eta   = np.random.rand()

    for i in range(D.N):
        value += coeff[i%space.NbaseD]*D(eta, i)*p/(T[1:-1][i + p] - T[1:-1][i])

    print(np.allclose(value, eva_1d.evaluate_d(space.t, space.p - 1, space.NbaseD, coeff, eta)))

True
True
True
True
True
True
True
True
True
True


In [8]:
# test evaluate_diffn
for cases in range(10):
    coeff = np.random.rand(space.NbaseN)

    value = 0.
    eta   = np.random.rand()

    for i in range(N.N):
        value += coeff[i%space.NbaseN]*N(eta, i, 1)

    print(np.allclose(value, eva_1d.evaluate_diffn(space.T, space.p, space.NbaseN, coeff, eta)))

True
True
True
True
True
True
True
True
True
True


## 2d

In [7]:
Nel        = [8, 6]                    # number of elements on logical domain
bc         = [False, False]            # boundary conditions (True: periodic, False: else)
p          = [3, 2]                    # spline degrees  

el_b       = [np.linspace(0., 1., Nel + 1) for Nel in Nel]                       # element boundaries
T          = [bsp.make_knots(el_b, p, bc) for el_b, p, bc in zip(el_b, p, bc)]   # knot vectors (for N functions)

nq_el      = [6, 6]                    # number of quadrature points per element for integrations over whole domain

spaces     = [spl.spline_space_1d(T, p, bc, nq_el) for T, p, bc, nq_el in zip(T, p, bc, nq_el)] # 1d B-spline spaces
t_space    = spl.tensor_spline_space(spaces)

N          = [bspline.Bspline(spl.T, spl.p)     for spl in spaces]     # N-splines for comparison
D          = [bspline.Bspline(spl.t, spl.p - 1) for spl in spaces]     # D-splines for comparison

In [57]:
# test evaluate_n_n
for cases in range(10):
    coeff = np.random.rand(t_space.NbaseN[0], t_space.NbaseN[1])

    value = 0.
    eta1  = np.random.rand()
    eta2  = np.random.rand()

    for i1 in range(N[0].N):
        for i2 in range(N[1].N):
            value += coeff[i1%t_space.NbaseN[0], i2%t_space.NbaseN[1]]*N[0](eta1, i1)*N[1](eta2, i2)

    
    if not np.allclose(value, eva_2d.evaluate_n_n(t_space.T[0], t_space.T[1], t_space.p[0], t_space.p[1], t_space.NbaseN[0], t_space.NbaseN[1], coeff, eta1, eta2)):
        raise ValueError

In [9]:
# test evaluate_d_n
for cases in range(10):
    coeff = np.random.rand(t_space.NbaseD[0], t_space.NbaseN[1])

    value = 0.
    eta1  = np.random.rand()
    eta2  = np.random.rand()

    for i1 in range(D[0].N):
        for i2 in range(N[1].N):
            value += coeff[i1%t_space.NbaseD[0], i2%t_space.NbaseN[1]]*D[0](eta1, i1)*N[1](eta2, i2)*p[0]/(T[0][1:-1][i1 + p[0]] - T[0][1:-1][i1])

    print(np.allclose(value, eva_2d.evaluate_d_n(t_space.t[0], t_space.T[1], t_space.p[0] - 1, t_space.p[1], t_space.NbaseD[0], t_space.NbaseN[1], coeff, eta1, eta2)))

True
True
True
True
True
True
True
True
True
True


In [10]:
# test evaluate_n_d
for cases in range(10):
    coeff = np.random.rand(t_space.NbaseN[0], t_space.NbaseD[1])

    value = 0.
    eta1  = np.random.rand()
    eta2  = np.random.rand()

    for i1 in range(N[0].N):
        for i2 in range(D[1].N):
            value += coeff[i1%t_space.NbaseN[0], i2%t_space.NbaseD[1]]*N[0](eta1, i1)*D[1](eta2, i2)*p[1]/(T[1][1:-1][i2 + p[1]] - T[1][1:-1][i2])

    print(np.allclose(value, eva_2d.evaluate_n_d(t_space.T[0], t_space.t[1], t_space.p[0], t_space.p[1] - 1, t_space.NbaseN[0], t_space.NbaseD[1], coeff, eta1, eta2)))

True
True
True
True
True
True
True
True
True
True


In [11]:
# test evaluate_d_d
for cases in range(10):
    coeff = np.random.rand(t_space.NbaseD[0], t_space.NbaseD[1])

    value = 0.
    eta1  = np.random.rand()
    eta2  = np.random.rand()

    for i1 in range(D[0].N):
        for i2 in range(D[1].N):
            value += coeff[i1%t_space.NbaseD[0], i2%t_space.NbaseD[1]]*D[0](eta1, i1)*D[1](eta2, i2)*p[0]/(T[0][1:-1][i1 + p[0]] - T[0][1:-1][i1])*p[1]/(T[1][1:-1][i2 + p[1]] - T[1][1:-1][i2])

    print(np.allclose(value, eva_2d.evaluate_d_d(t_space.t[0], t_space.t[1], t_space.p[0] - 1, t_space.p[1] - 1, t_space.NbaseD[0], t_space.NbaseD[1], coeff, eta1, eta2)))

True
True
True
True
True
True
True
True
True
True


In [12]:
# test evaluate_diffn_n
for cases in range(10):
    coeff = np.random.rand(t_space.NbaseN[0], t_space.NbaseN[1])

    value = 0.
    eta1  = np.random.rand()
    eta2  = np.random.rand()

    for i1 in range(N[0].N):
        for i2 in range(N[1].N):
            value += coeff[i1%t_space.NbaseN[0], i2%t_space.NbaseN[1]]*N[0](eta1, i1, 1)*N[1](eta2, i2)

    print(np.allclose(value, eva_2d.evaluate_diffn_n(t_space.T[0], t_space.T[1], t_space.p[0], t_space.p[1], t_space.NbaseN[0], t_space.NbaseN[1], coeff, eta1, eta2)))

True
True
True
True
True
True
True
True
True
True


In [13]:
# test evaluate_n_diffn
for cases in range(10):
    coeff = np.random.rand(t_space.NbaseN[0], t_space.NbaseN[1])

    value = 0.
    eta1  = np.random.rand()
    eta2  = np.random.rand()

    for i1 in range(N[0].N):
        for i2 in range(N[1].N):
            value += coeff[i1%t_space.NbaseN[0], i2%t_space.NbaseN[1]]*N[0](eta1, i1)*N[1](eta2, i2, 1)

    print(np.allclose(value, eva_2d.evaluate_n_diffn(t_space.T[0], t_space.T[1], t_space.p[0], t_space.p[1], t_space.NbaseN[0], t_space.NbaseN[1], coeff, eta1, eta2)))

True
True
True
True
True
True
True
True
True
True


## 3d

In [2]:
Nel        = [8, 6, 10]                    # number of elements on logical domain
bc         = [True, True, True]         # boundary conditions (True: periodic, False: else)
p          = [3, 2, 1]                     # spline degrees  

el_b       = [np.linspace(0., 1., Nel + 1) for Nel in Nel]                       # element boundaries
T          = [bsp.make_knots(el_b, p, bc) for el_b, p, bc in zip(el_b, p, bc)]   # knot vectors (for N functions)

nq_el      = [6, 6, 6]                     # number of quadrature points per element for integrations over whole domain

spaces     = [spl.spline_space_1d(T, p, bc, nq_el) for T, p, bc, nq_el in zip(T, p, bc, nq_el)] # 1d B-spline spaces
t_space    = spl.tensor_spline_space(spaces)

N          = [bspline.Bspline(spl.T, spl.p)     for spl in spaces]     # N-splines for comparison
D          = [bspline.Bspline(spl.t, spl.p - 1) for spl in spaces]     # D-splines for comparison

In [3]:
# test evaluate_n_n_n
for cases in range(10):
    coeff = np.random.rand(t_space.NbaseN[0], t_space.NbaseN[1], t_space.NbaseN[2])

    value = 0.
    eta1  = np.random.rand()
    eta2  = np.random.rand()
    eta3  = np.random.rand()

    for i1 in range(N[0].N):
        for i2 in range(N[1].N):
            for i3 in range(N[2].N):
                value += coeff[i1%t_space.NbaseN[0], i2%t_space.NbaseN[1], i3%t_space.NbaseN[2]]*N[0](eta1, i1)*N[1](eta2, i2)*N[2](eta3, i3)

    print(np.allclose(value, eva_3d.evaluate_n_n_n(t_space.T[0], t_space.T[1], t_space.T[2], t_space.p[0], t_space.p[1], t_space.p[2], t_space.NbaseN[0], t_space.NbaseN[1], t_space.NbaseN[2], coeff, eta1, eta2, eta3)))

True
True
True
True
True
True
True
True
True
True


In [4]:
# test evaluate_d_n_n
for cases in range(10):
    coeff = np.random.rand(t_space.NbaseD[0], t_space.NbaseN[1], t_space.NbaseN[2])

    value = 0.
    eta1  = np.random.rand()
    eta2  = np.random.rand()
    eta3  = np.random.rand()

    for i1 in range(D[0].N):
        for i2 in range(N[1].N):
            for i3 in range(N[2].N):
                value += coeff[i1%t_space.NbaseD[0], i2%t_space.NbaseN[1], i3%t_space.NbaseN[2]]*D[0](eta1, i1)*N[1](eta2, i2)*N[2](eta3, i3)*p[0]/(T[0][1:-1][i1 + p[0]] - T[0][1:-1][i1])

    print(np.allclose(value, eva_3d.evaluate_d_n_n(t_space.t[0], t_space.T[1], t_space.T[2], t_space.p[0] - 1, t_space.p[1], t_space.p[2], t_space.NbaseD[0], t_space.NbaseN[1], t_space.NbaseN[2], coeff, eta1, eta2, eta3)))

True
True
True
True
True
True
True
True
True
True


In [5]:
# test evaluate_n_d_n
for cases in range(10):
    coeff = np.random.rand(t_space.NbaseN[0], t_space.NbaseD[1], t_space.NbaseN[2])

    value = 0.
    eta1  = np.random.rand()
    eta2  = np.random.rand()
    eta3  = np.random.rand()

    for i1 in range(N[0].N):
        for i2 in range(D[1].N):
            for i3 in range(N[2].N):
                value += coeff[i1%t_space.NbaseN[0], i2%t_space.NbaseD[1], i3%t_space.NbaseN[2]]*N[0](eta1, i1)*D[1](eta2, i2)*N[2](eta3, i3)*p[1]/(T[1][1:-1][i2 + p[1]] - T[1][1:-1][i2])

    print(np.allclose(value, eva_3d.evaluate_n_d_n(t_space.T[0], t_space.t[1], t_space.T[2], t_space.p[0], t_space.p[1] - 1, t_space.p[2], t_space.NbaseN[0], t_space.NbaseD[1], t_space.NbaseN[2], coeff, eta1, eta2, eta3)))

True
True
True
True
True
True
True
True
True
True


In [6]:
# test evaluate_n_n_d
for cases in range(10):
    coeff = np.random.rand(t_space.NbaseN[0], t_space.NbaseN[1], t_space.NbaseD[2])

    value = 0.
    eta1  = np.random.rand()
    eta2  = np.random.rand()
    eta3  = np.random.rand()

    for i1 in range(N[0].N):
        for i2 in range(N[1].N):
            for i3 in range(D[2].N):
                value += coeff[i1%t_space.NbaseN[0], i2%t_space.NbaseN[1], i3%t_space.NbaseD[2]]*N[0](eta1, i1)*N[1](eta2, i2)*D[2](eta3, i3)*p[2]/(T[2][1:-1][i3 + p[2]] - T[2][1:-1][i3])

    print(np.allclose(value, eva_3d.evaluate_n_n_d(t_space.T[0], t_space.T[1], t_space.t[2], t_space.p[0], t_space.p[1], t_space.p[2] - 1, t_space.NbaseN[0], t_space.NbaseN[1], t_space.NbaseD[2], coeff, eta1, eta2, eta3)))

True
True
True
True
True
True
True
True
True
True


In [7]:
# test evaluate_diffn_n_n
for cases in range(10):
    coeff = np.random.rand(t_space.NbaseN[0], t_space.NbaseN[1], t_space.NbaseN[2])

    value = 0.
    eta1  = np.random.rand()
    eta2  = np.random.rand()
    eta3  = np.random.rand()

    for i1 in range(N[0].N):
        for i2 in range(N[1].N):
            for i3 in range(N[2].N):
                value += coeff[i1%t_space.NbaseN[0], i2%t_space.NbaseN[1], i3%t_space.NbaseN[2]]*N[0](eta1, i1, 1)*N[1](eta2, i2)*N[2](eta3, i3)

    print(np.allclose(value, eva_3d.evaluate_diffn_n_n(t_space.T[0], t_space.T[1], t_space.T[2], t_space.p[0], t_space.p[1], t_space.p[2], t_space.NbaseN[0], t_space.NbaseN[1], t_space.NbaseN[2], coeff, eta1, eta2, eta3)))

True
True
True
True
True
True
True
True
True
True


In [8]:
# test evaluate_n_diffn_n
for cases in range(10):
    coeff = np.random.rand(t_space.NbaseN[0], t_space.NbaseN[1], t_space.NbaseN[2])

    value = 0.
    eta1  = np.random.rand()
    eta2  = np.random.rand()
    eta3  = np.random.rand()

    for i1 in range(N[0].N):
        for i2 in range(N[1].N):
            for i3 in range(N[2].N):
                value += coeff[i1%t_space.NbaseN[0], i2%t_space.NbaseN[1], i3%t_space.NbaseN[2]]*N[0](eta1, i1)*N[1](eta2, i2, 1)*N[2](eta3, i3)

    print(np.allclose(value, eva_3d.evaluate_n_diffn_n(t_space.T[0], t_space.T[1], t_space.T[2], t_space.p[0], t_space.p[1], t_space.p[2], t_space.NbaseN[0], t_space.NbaseN[1], t_space.NbaseN[2], coeff, eta1, eta2, eta3)))

True
True
True
True
True
True
True
True
True
True


In [9]:
# test evaluate_n_n_diffn
for cases in range(10):
    coeff = np.random.rand(t_space.NbaseN[0], t_space.NbaseN[1], t_space.NbaseN[2])

    value = 0.
    eta1  = np.random.rand()
    eta2  = np.random.rand()
    eta3  = np.random.rand()

    for i1 in range(N[0].N):
        for i2 in range(N[1].N):
            for i3 in range(N[2].N):
                value += coeff[i1%t_space.NbaseN[0], i2%t_space.NbaseN[1], i3%t_space.NbaseN[2]]*N[0](eta1, i1)*N[1](eta2, i2)*N[2](eta3, i3, 1)

    print(np.allclose(value, eva_3d.evaluate_n_n_diffn(t_space.T[0], t_space.T[1], t_space.T[2], t_space.p[0], t_space.p[1], t_space.p[2], t_space.NbaseN[0], t_space.NbaseN[1], t_space.NbaseN[2], coeff, eta1, eta2, eta3)))

True
True
True
True
True
True
True
True
True
True


In [41]:
# test evaluate_n_d_d
for cases in range(10):
    coeff = np.random.rand(t_space.NbaseN[0], t_space.NbaseD[1], t_space.NbaseD[2])

    value = 0.
    eta1  = np.random.rand()
    eta2  = np.random.rand()
    eta3  = np.random.rand()

    for i1 in range(N[0].N):
        for i2 in range(D[1].N):
            for i3 in range(D[2].N):
                value += coeff[i1%t_space.NbaseN[0], i2%t_space.NbaseD[1], i3%t_space.NbaseD[2]]*N[0](eta1, i1)*D[1](eta2, i2)*D[2](eta3, i3)*p[1]/(T[1][1:-1][i2 + p[1]] - T[1][1:-1][i2])*p[2]/(T[2][1:-1][i3 + p[2]] - T[2][1:-1][i3])

    print(np.allclose(value, eva_3d.evaluate_n_d_d(t_space.T[0], t_space.t[1], t_space.t[2], t_space.p[0], t_space.p[1] - 1, t_space.p[2] - 1, t_space.NbaseN[0], t_space.NbaseD[1], t_space.NbaseD[2], coeff, eta1, eta2, eta3)))

True
True
True
True
True
True
True
True
True
True


In [43]:
# test evaluate_d_n_d
for cases in range(10):
    coeff = np.random.rand(t_space.NbaseD[0], t_space.NbaseN[1], t_space.NbaseD[2])

    value = 0.
    eta1  = np.random.rand()
    eta2  = np.random.rand()
    eta3  = np.random.rand()

    for i1 in range(D[0].N):
        for i2 in range(N[1].N):
            for i3 in range(D[2].N):
                value += coeff[i1%t_space.NbaseD[0], i2%t_space.NbaseN[1], i3%t_space.NbaseD[2]]*D[0](eta1, i1)*N[1](eta2, i2)*D[2](eta3, i3)*p[0]/(T[0][1:-1][i1 + p[0]] - T[0][1:-1][i1])*p[2]/(T[2][1:-1][i3 + p[2]] - T[2][1:-1][i3])

    print(np.allclose(value, eva_3d.evaluate_d_n_d(t_space.t[0], t_space.T[1], t_space.t[2], t_space.p[0] - 1, t_space.p[1], t_space.p[2] - 1, t_space.NbaseD[0], t_space.NbaseN[1], t_space.NbaseD[2], coeff, eta1, eta2, eta3)))

True
True
True
True
True
True
True
True
True
True


In [45]:
# test evaluate_d_d_n
for cases in range(10):
    coeff = np.random.rand(t_space.NbaseD[0], t_space.NbaseD[1], t_space.NbaseN[2])

    value = 0.
    eta1  = np.random.rand()
    eta2  = np.random.rand()
    eta3  = np.random.rand()

    for i1 in range(D[0].N):
        for i2 in range(D[1].N):
            for i3 in range(N[2].N):
                value += coeff[i1%t_space.NbaseD[0], i2%t_space.NbaseD[1], i3%t_space.NbaseN[2]]*D[0](eta1, i1)*D[1](eta2, i2)*N[2](eta3, i3)*p[0]/(T[0][1:-1][i1 + p[0]] - T[0][1:-1][i1])*p[1]/(T[1][1:-1][i2 + p[1]] - T[1][1:-1][i2])

    print(np.allclose(value, eva_3d.evaluate_d_d_n(t_space.t[0], t_space.t[1], t_space.T[2], t_space.p[0] - 1, t_space.p[1] - 1, t_space.p[2], t_space.NbaseD[0], t_space.NbaseD[1], t_space.NbaseN[2], coeff, eta1, eta2, eta3)))

True
True
True
True
True
True
True
True
True
True


In [47]:
# test evaluate_d_d_d
for cases in range(10):
    coeff = np.random.rand(t_space.NbaseD[0], t_space.NbaseD[1], t_space.NbaseD[2])

    value = 0.
    eta1  = np.random.rand()
    eta2  = np.random.rand()
    eta3  = np.random.rand()

    for i1 in range(D[0].N):
        for i2 in range(D[1].N):
            for i3 in range(D[2].N):
                value += coeff[i1%t_space.NbaseD[0], i2%t_space.NbaseD[1], i3%t_space.NbaseD[2]]*D[0](eta1, i1)*D[1](eta2, i2)*D[2](eta3, i3)*p[0]/(T[0][1:-1][i1 + p[0]] - T[0][1:-1][i1])*p[1]/(T[1][1:-1][i2 + p[1]] - T[1][1:-1][i2])*p[2]/(T[2][1:-1][i3 + p[2]] - T[2][1:-1][i3])

    print(np.allclose(value, eva_3d.evaluate_d_d_d(t_space.t[0], t_space.t[1], t_space.t[2], t_space.p[0] - 1, t_space.p[1] - 1, t_space.p[2] - 1, t_space.NbaseD[0], t_space.NbaseD[1], t_space.NbaseD[2], coeff, eta1, eta2, eta3)))

True
True
True
True
True
True
True
True
True
True
